## Admisión en la universidad

In [5]:
import pandas as pd
import numpy as np

# import file
df = pd.read_csv('../../res/tp1/binary.csv', sep=',') # columns: admit, gre, gpa, rank; 399 rows
row_count = df.shape[0]
rank_count = df['rank'].value_counts().sort_values()
ranks = np.sort(rank_count.index.values)
first_level_fields = {'gre': ['>= 500', '<  500'], 'gpa': ['>= 3', '<  3']}

In [6]:
rank_probabilities = rank_count.apply(lambda x: (x + 1) / (row_count + len(ranks)))  # P(rank_i)
print(f'Probability for each rank:\n{rank_probabilities}')
fields_rank_probabilities = {key: {} for key in first_level_fields.keys()} # {'gre':{}, 'gpa':{}}
for rank in ranks:
    rank_rows = df.query(f'rank == {rank}')
    for field, rng in first_level_fields.items():
        fields_rank_probabilities[field][rank] = []
        upper_matching = rank_rows.query(f'{field} {rng[0]}') # e.g. 'gre >= 500'
        # Chequear si es necesario Laplace
        fields_rank_probabilities[field][rank].append((len(upper_matching) + 1) / (len(rank_rows) + 2)) # len(gre >= 500 and rank) / len(rank)
        fields_rank_probabilities[field][rank].append((len(rank_rows) - len(upper_matching) + 1) / (len(rank_rows) + 2)) # gre < 500

# {'gre': {1: [upper, lower], 2: [upper, lower], ...}, 'gpa': {1: [upper, lower], 2: [upper, lower], ...}}
gre_table = pd.DataFrame.from_dict(fields_rank_probabilities['gre'], orient='index', columns=first_level_fields['gre'])
gpa_table = pd.DataFrame.from_dict(fields_rank_probabilities['gpa'], orient='index', columns=first_level_fields['gpa'])
print(f'\nGRE:\n{gre_table}')
print(f'\nGPA:\n{gpa_table}')

Probability for each rank:
1    0.153465
4    0.168317
3    0.301980
2    0.376238
Name: rank, dtype: float64

GRE:
     >= 500    <  500
1  0.809524  0.190476
2  0.810458  0.189542
3  0.788618  0.211382
4  0.782609  0.217391

GPA:
       >= 3      <  3
1  0.857143  0.142857
2  0.823529  0.176471
3  0.829268  0.170732
4  0.797101  0.202899


In [7]:
# Necesitamos P(admit | rank, GRE, GPA) y probabilidad conjunta P(X1, X2, ..., Xn)

admission_probabilities = {}
for rank in ranks:
    # gre_row = gre_table.loc[rank]
    # gpa_row = gpa_table.loc[rank]
    for gre_class in gre_table.columns:
        # p_gre = gre_row[gre_class]
        for gpa_class in gpa_table.columns:
            row_criteria = f'rank == {rank} and gre {gre_class} and gpa {gpa_class}'
            criteria_rows = df.query(row_criteria)
            row_criteria_admissions = criteria_rows[criteria_rows.admit == 1]
            admission_probabilities[row_criteria] = []
            # Laplace!
            admission_probabilities[row_criteria].append((len(row_criteria_admissions) + 1) / (len(criteria_rows) + 2))
            admission_probabilities[row_criteria].append((len(criteria_rows) - len(row_criteria_admissions) + 1) / (len(criteria_rows) + 2))
            # p_gpa = gpa_row[gpa_class]
            # p = p_gre * p_gpa
            # admission_probabilities[row_criteria].append(p)
            print(f'Rank: {rank}; GRE: {gre_class}; GPA: {gpa_class}; rows: {len(criteria_rows):03}; admission probability: {admission_probabilities[row_criteria][0]}')

admission_table = pd.DataFrame.from_dict(admission_probabilities, orient='index', columns=['1', '0'])
print(f'\nAdmissions:\n{admission_table}')

Rank: 1; GRE: >= 500; GPA: >= 3; rows: 047; admission probability: 0.5510204081632653
Rank: 1; GRE: >= 500; GPA: <  3; rows: 003; admission probability: 0.8
Rank: 1; GRE: <  500; GPA: >= 3; rows: 006; admission probability: 0.5
Rank: 1; GRE: <  500; GPA: <  3; rows: 005; admission probability: 0.2857142857142857
Rank: 2; GRE: >= 500; GPA: >= 3; rows: 104; admission probability: 0.42452830188679247
Rank: 2; GRE: >= 500; GPA: <  3; rows: 019; admission probability: 0.19047619047619047
Rank: 2; GRE: <  500; GPA: >= 3; rows: 021; admission probability: 0.21739130434782608
Rank: 2; GRE: <  500; GPA: <  3; rows: 007; admission probability: 0.4444444444444444
Rank: 3; GRE: >= 500; GPA: >= 3; rows: 085; admission probability: 0.25287356321839083
Rank: 3; GRE: >= 500; GPA: <  3; rows: 011; admission probability: 0.38461538461538464
Rank: 3; GRE: <  500; GPA: >= 3; rows: 016; admission probability: 0.2222222222222222
Rank: 3; GRE: <  500; GPA: <  3; rows: 009; admission probability: 0.0909090909

In [10]:
def admission_probability(person):
    # person['gre'] = '>= 500' if person['gre'] >= 500 else '<  500'
    # person['gpa'] = '>= 3' if person['gpa'] >= 3 else '<  3'
    # rank_prob = rank_probabilities.loc[person['rank']]
    gre_index = 0 if person['gre'] >= 500 else 1
    gpa_index = 0 if person['gpa'] >= 3 else 1
    gre_prob = gre_table.loc[person['rank']].values[gre_index]
    gpa_prob = gpa_table.loc[person['rank']].values[gpa_index]
    cond = f'rank == {person["rank"]} and gre {first_level_fields["gre"][gre_index]} and gpa {first_level_fields["gpa"][gpa_index]}'
    admission_prob = admission_table.loc[cond]
    # conj_prob = rank_prob * gre_prob * gpa_prob * admission_prob.values[0]
    # print(conj_prob)
    print(f'La probabilidad de que una persona que fue a una escuela de rango 2, tenga GRE = 450 y GPA = 3.5 sea admitida en la universidad es de {(admission_prob.values[0] * 100):.5} %')
    # print(f'{rank_prob} {gre_prob} {gpa_prob}')

person = {'rank': 2, 'gre': 450, 'gpa': 3.5}
admission_probability(person)

La probabilidad de que una persona que fue a una escuela de rango 2, tenga GRE = 450 y GPA = 3.5 sea admitida en la universidad es de 21.739 %


In [15]:
# P(admit = 0 | rank = 1)
#   = P(admit, rank) / P(rank) 
#   = ∑_gre ∑_gpa P(admit, gre, gpa, rank) 
#   = ∑_gre ∑_gpa P(admit | gre, gpa, rank) * P(gre | rank) * P(gpa | rank) * P(rank)

def non_admission_probability(rank):
    suma = 0
    for gre_rng in first_level_fields['gre']:
        gre_prob = gre_table.loc[rank][gre_rng] # P('gre' = X | 'rank' = rank)
        for gpa_rng in first_level_fields['gpa']:
            gpa_prob = gpa_table.loc[rank][gpa_rng] # P('gpa' = Y | 'rank' = rank)
            criteria = f'rank == {rank} and gre {gre_rng} and gpa {gpa_rng}'
            non_admission_prob = admission_table.loc[criteria].values[1] # P(admit = 0 | 'gre' = X, 'gpa' = Y, 'rank' = rank)
            # print(non_admission_prob)
            suma += gre_prob * gpa_prob * non_admission_prob * rank_probabilities.loc[rank]
    suma /= rank_probabilities.loc[rank]
    print(f'La probabilidad de que una persona que proviene de una escuela con rango {rank} no haya sido admitida en la universidad es de {(suma * 100):.5} %')

non_admission_probability(1)
non_admission_probability(2)
non_admission_probability(3)
non_admission_probability(4)

La probabilidad de que una persona que proviene de una escuela con rango 1 no haya sido admitida en la universidad es de 43.574 %
La probabilidad de que una persona que proviene de una escuela con rango 2 no haya sido admitida en la universidad es de 64.061 %
La probabilidad de que una persona que proviene de una escuela con rango 3 no haya sido admitida en la universidad es de 74.061 %
La probabilidad de que una persona que proviene de una escuela con rango 4 no haya sido admitida en la universidad es de 78.484 %


### ¿Dónde está el aprendizaje?
En este ejercicio, el aprendizaje compre únicamente la etapa de aprendizaje paramétrico, ya que la estructura de la red nos es dada y por lo tanto no hace falta inferirla mediante algoritmos como K2. Dado un conjunto de entrenamiento (la "experiencia") se obtuvieron distintas probabilidades (a priori y condicionales) con el fin de evaluar el comportamiento y lo que infieren dichos datos. En base a lo que se quiera calcular, se implementaron distintas funciones o tareas con ese mismo fin. Si aumentaran la cantidad de registros, el programa se adaptaría a dichos cambios.